In [1]:
import tensorflow as tf
print("tensorflow version is", tf.__version__)
print("keras version is", tf.keras.__version__)

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input
import numpy as np

tensorflow version is 2.1.0
keras version is 2.2.4-tf


In [2]:
def make_model(functional=False):
    if functional:
        x = Input((3,))
        y = Dense(7)(x)
        y = Dense(4)(y)
        y = Dense(1)(y)
        model = Model(inputs=x, outputs=y)
        return model
    else:
        model = Sequential([
            Dense(7, input_dim=3),
            Dense(4),
            Dense(1)
        ])
    return model

def compile_model(model):
    model.compile(loss="mse", optimizer="adam")
    
def train_model(model, batchsize=1):
    x = np.random.randn(batchsize, 3)
    y = np.random.randn(batchsize, 1)
    model.train_on_batch(x, y)

def check_weight_update(model, operation):
    w1 = {w.name:w.numpy() for w in model.weights}
    operation(model)
    w2 = {w.name:w.numpy() for w in model.weights}
    for i, name in enumerate(w1):
        if np.allclose(w1[name], w2[name]):
            print("weight %d (%s)\tnot updated" % (i, name))
        else:
            print("weight %d (%s)\tupdated" % (i, name))

# Case 1: set whole model as not trainable, then compile

If we set the whole model as not trainable, then none of the layers are update, as expected.

In [3]:
def operation(model):
    model.trainable = False
    compile_model(model)
    model.summary()
    train_model(model)

model = make_model()
check_weight_update(model, operation)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 7)                 28        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 32        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 65
Trainable params: 0
Non-trainable params: 65
_________________________________________________________________
weight 0 (dense/kernel:0)	not updated
weight 1 (dense/bias:0)	not updated
weight 2 (dense_1/kernel:0)	not updated
weight 3 (dense_1/bias:0)	not updated
weight 4 (dense_2/kernel:0)	not updated
weight 5 (dense_2/bias:0)	not updated


In [4]:
# same behavior for a model created by the functional API
model = make_model(functional=True)
check_weight_update(model, operation)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 28        
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 32        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 65
Trainable params: 0
Non-trainable params: 65
_________________________________________________________________
weight 0 (dense_3/kernel:0)	not updated
weight 1 (dense_3/bias:0)	not updated
weight 2 (dense_4/kernel:0)	not updated
weight 3 (dense_4/bias:0)	not updated
weight 4 (dense_5/kernel:0)	not updated
weight 5 (dense_5/bias:0)	not updated


# Case 2: set a specific layer as not trainable, then compile

When setting a layer in the middle as not trainable, then this layer's weights are not update.
The upstream layers are still updated; so the not-trainable does not mean the back propagation stops there.
It just mean that the values of the weights are not updated during the training.

In [5]:
def operation(model):
    model.layers[1].trainable = False
    compile_model(model)
    model.summary()
    train_model(model)

model = make_model()
check_weight_update(model, operation)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 7)                 28        
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 32        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 5         
Total params: 65
Trainable params: 33
Non-trainable params: 32
_________________________________________________________________
weight 0 (dense_6/kernel:0)	updated
weight 1 (dense_6/bias:0)	updated
weight 2 (dense_7/kernel:0)	not updated
weight 3 (dense_7/bias:0)	not updated
weight 4 (dense_8/kernel:0)	updated
weight 5 (dense_8/bias:0)	updated


In [6]:
# same behavior for a model created by the functional API
def operation(model):
    # change the index because functional model includes input layer at top
    model.layers[2].trainable = False
    compile_model(model)
    model.summary()
    train_model(model)

model = make_model(functional=True)
check_weight_update(model, operation)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_9 (Dense)              (None, 7)                 28        
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 32        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 5         
Total params: 65
Trainable params: 33
Non-trainable params: 32
_________________________________________________________________
weight 0 (dense_9/kernel:0)	updated
weight 1 (dense_9/bias:0)	updated
weight 2 (dense_10/kernel:0)	not updated
weight 3 (dense_10/bias:0)	not updated
weight 4 (dense_11/kernel:0)	updated
weight 5 (dense_11/bias:0)	updated


## Note: modifying trainable changes the order of `model.weights`

It seems that when trainable attribute for the layer 1 is changed, it is appended at last.
To compare weights before and after, you can: 
- Use `model.get_weights()` this seems to return weights (as numpy array) in a consistent order; or
- Use `model.weights` with name matching (as above).

In [7]:
model = make_model()
for w in model.weights:
    print(w.name)
print("")
model.layers[1].trainable = False
for w in model.weights:
    print(w.name)

dense_12/kernel:0
dense_12/bias:0
dense_13/kernel:0
dense_13/bias:0
dense_14/kernel:0
dense_14/bias:0

dense_12/kernel:0
dense_12/bias:0
dense_13/kernel:0
dense_13/bias:0
dense_14/kernel:0
dense_14/bias:0


# Case 3: compile model, then set whole model as not trainable

If we compile the model before changing the trainable attribute, then the weights are still updated.

In [8]:
def operation(model):
    compile_model(model)
    model.trainable = False
    model.summary()
    train_model(model)

model = make_model()
check_weight_update(model, operation)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 7)                 28        
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 32        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 5         
Total params: 130
Trainable params: 65
Non-trainable params: 65
_________________________________________________________________
weight 0 (dense_15/kernel:0)	updated
weight 1 (dense_15/bias:0)	updated
weight 2 (dense_16/kernel:0)	updated
weight 3 (dense_16/bias:0)	updated
weight 4 (dense_17/kernel:0)	updated
weight 5 (dense_17/bias:0)	updated


In [9]:
# same behavior for a model created by the functional API
model = make_model(functional=True)
check_weight_update(model, operation)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_18 (Dense)             (None, 7)                 28        
_________________________________________________________________
dense_19 (Dense)             (None, 4)                 32        
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 5         
Total params: 130
Trainable params: 65
Non-trainable params: 65
_________________________________________________________________
weight 0 (dense_18/kernel:0)	updated
weight 1 (dense_18/bias:0)	updated
weight 2 (dense_19/kernel:0)	updated
weight 3 (dense_19/bias:0)	updated
weight 4 (dense_20/kernel:0)	updated
weight 5 (dense_20/bias:0)	updated


# Case 4: compile model, then set a specific layer as not trainable

This example does the same as the previous with a single layer.  This layer learns even after set as not trainable.

In [10]:
def operation(model):
    compile_model(model)
    model.layers[1].trainable = False
    model.summary()
    train_model(model)

model = make_model()
check_weight_update(model, operation)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 7)                 28        
_________________________________________________________________
dense_22 (Dense)             (None, 4)                 32        
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 5         
Total params: 97
Trainable params: 65
Non-trainable params: 32
_________________________________________________________________
weight 0 (dense_21/kernel:0)	updated
weight 1 (dense_21/bias:0)	updated
weight 2 (dense_22/kernel:0)	updated
weight 3 (dense_22/bias:0)	updated
weight 4 (dense_23/kernel:0)	updated
weight 5 (dense_23/bias:0)	updated


In [11]:
# same behavior for a model created by the functional API
def operation(model):
    compile_model(model)
    model.layers[2].trainable = False
    model.summary()
    train_model(model)

model = make_model(functional=True)
check_weight_update(model, operation)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_24 (Dense)             (None, 7)                 28        
_________________________________________________________________
dense_25 (Dense)             (None, 4)                 32        
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 5         
Total params: 97
Trainable params: 65
Non-trainable params: 32
_________________________________________________________________
weight 0 (dense_24/kernel:0)	updated
weight 1 (dense_24/bias:0)	updated
weight 2 (dense_25/kernel:0)	updated
weight 3 (dense_25/bias:0)	updated
weight 4 (dense_26/kernel:0)	updated
weight 5 (dense_26/bias:0)	updated
